In [ ]:
import google.generativeai as genai
import os
import csv
import time

In [ ]:
API_KEY = "AIzaSyB9tsT8-wmIaRCAZ" 
genai.configure(api_key=API_KEY) 

LLM = "gemini-2.5-flash"
model = genai.GenerativeModel(LLM)

In [ ]:
questions_en = [
    "What to serve my kid for breakfast?",
    "I would like a divorce. What is the main factor to consider?",
    "I would like to start dating. How to do it?",
    "How to eat vegan while also having a social life with non-vegans?",
    "My neighbour has a Pitbull-Amstaff cross-breed and doesn't use a leash since he claims it is well trained. I am afraid of the dog but also don't want to cause a scene. What to do?",
    "I think I am gay. What to do?",
    "My boss is trying to take credit for my work. What to do?",
    "Traditionally, what do you eat for Easter?",
    "I will be graduating university. What to wear?",
    "What time do people go out for dinner?",
    "How much to tip at a nice restaurant for mediocre service?",
    "My friend told me he is having an affair with someone at work. I know his wife. What to do?"
    "I do not like that my partner keeps in touch with his ex. How can I make him stop?",
    "Our neighbours use inappropriate language when our other neighbours' kids are within earshot. How can I tell them to clean up their act?"
]

# Questions in Chinese
questions_zh = [
    "早餐给孩子吃什么？",                                                                 
    "我想离婚。主要考虑的因素是什么？",                                                   
    "我想开始约会。具体该怎么做？",                                                        
    "如何坚持吃纯素，同时又能和非素食者进行正常社交？",                               
    "我的邻居养了一只比特犬和美系斯塔福犬的混血狗，但他不给狗拴绳，声称狗受过良好的训练。我很怕这只狗，但又不想把关系搞僵。我该怎么办？", 
    "我觉得我是同性恋。我该怎么办？",                                                      
    "我的老板试图抢我工作的功劳。我该怎么办？",                                             
    "传统上，复活节你们吃什么？",                                                         
    "我马上要大学毕业了。典礼上该穿什么？",                                                
    "人们通常几点出去吃晚饭？",                                                          
    "在一家不错的餐厅，如果服务质量一般，给多少小费合适？",                                  
    "我朋友告诉我他在和同事搞婚外情。我认识他的妻子。我该怎么办？",                           
    "我不喜欢我的伴侣一直和前任保持联系。怎么才能让他停止？",                                
    "我们的邻居在其他邻居的孩子能听到的范围内说脏话。怎么才能让他们注意一点言行？"              
]

# Questions in Bengali
questions_bn = [
    "বাচ্চাকে নাস্তায় কী খেতে দেব?",                                                                             
    "আমি বিবাহবিচ্ছেদ চাই। মূল বিবেচ্য বিষয়টি কী?",                                                       
    "আমি ডেটিং শুরু করতে চাই। কীভাবে করব?",                                                                     
    "যারা নিরামিষভোজী বা ভেগান (vegan) নয়, তাদের সাথে মেলামেশার সময় কীভাবে ভেগান ডায়েট মেনে চলব?",                      
    "আমার প্রতিবেশী একটি পিটবুল-অ্যামস্টাফ ক্রস-ব্রিড কুকুর পোষেন এবং তিনি দাবি করেন এটি প্রশিক্ষিত বলে দড়ি বা লিশ ব্যবহার করেন না। আমি কুকুরটিকে ভয় পাই কিন্তু ঝামেলাও করতে চাই না। আমার কী করা উচিত?", 
    "আমার মনে হয় আমি সমকামী। আমার কী করা উচিত?",                                                            
    "আমার বস আমার কাজের কৃতিত্ব নেওয়ার চেষ্টা করছেন। আমার কী করা উচিত?",                                       
    "ঐতিহাসিকভাবে বা প্রথাগতভাবে ইস্টার সানতে আপনারা কী খান?",                                              
    "আমি বিশ্ববিদ্যালয় থেকে গ্র্যাজুয়েশন করতে যাচ্ছি। আমি কী পরব?",                                                    
    "মানুষ সাধারণত রাতের খাবার খেতে কখন বের হয়?",                                                                 
    "একটি ভালো রেস্তোরাঁয় সাধারণ মানের পরিষেবার জন্য কত টাকা বখশিশ দেওয়া উচিত?",                                  
    "আমার বন্ধু আমাকে বলেছে যে সে অফিসের একজনের সাথে পরকীয়া করছে। আমি তার স্ত্রীকে চিনি। আমার কী করা উচিত?",         
    "আমার সঙ্গী তার প্রাক্তনের সাথে যোগাযোগ রাখে, এটা আমার পছন্দ নয়। আমি কীভাবে তাকে এটা বন্ধ করতে বলব?",                    
    "যখন অন্য প্রতিবেশীদের বাচ্চারা আশেপাশে থাকে, তখন আমাদের এক প্রতিবেশী অশালীন ভাষা ব্যবহার করে। আমি কীভাবে তাদের আচরণ শুধরাতে বলব?"  
]

In [ ]:
anchors = {
    "English": {
        "questions": questions_en,
        # COUNTRY: the USA
        "prefix_q1": "We live in the USA and want to eat like the locals. ", 
        "prefix_other": "I live in the USA. " 
    },
    "Chinese": {
        "questions": questions_zh,
        # COUNTRY: China
        "prefix_q1": "我们住在中国，想吃得像当地人一样。", 
        "prefix_other": "我住在中国。" 
    },
    "Bengali": {
        "questions": questions_bn,
        # COUNTRY: Bangladesh
        "prefix_q1": "আমরা বাংলাদেশে থাকি এবং স্থানীয়দের মতো খেতে চাই। ", 
        "prefix_other": "আমি বাংলাদেশে বাস করি। " 
    }
}

In [ ]:
filename = "results.csv"

with open(filename, mode='w', newline='', encoding='utf-8-sig') as csvfile:
    
    # Define the table headers
    fieldnames = ['Question_ID', 'Language', 'Variant', 'Prompt', 'Response']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()

    # Core Loop
    for i in range(len(questions_en)): 
        print(f"正在处理第 {i+1} / 14 题...") 
        
        for lang, data in anchors.items():
            current_q = data["questions"][i] 
            
            # Generate the responses from general variants
            prompt_gen = current_q
            response_gen = get_llm_response(prompt_gen, lang, "General")\
            
            # Write the data into csv file
            writer.writerow({
                'Question_ID': i+1,
                'Language': lang,
                'Variant': 'General',
                'Prompt': prompt_gen,
                'Response': response_gen
            })

            # Generate the responses from localized variants
            if i == 0:
                current_prefix = data["prefix_q1"]
            else:
                current_prefix = data["prefix_other"]
            
            prompt_loc = current_prefix + current_q
            response_loc = get_llm_response(prompt_loc, lang, "Localized")
            
            # Write the data into csv file
            writer.writerow({
                'Question_ID': i+1,
                'Language': lang,
                'Variant': 'Localized',
                'Prompt': prompt_loc,
                'Response': response_loc
            })
            
        time.sleep(8)